# Assignment 1: Linear Programming - The Case of Filatoi Riuniti

## import packages

In [17]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

## (b)

## import problem data

In [2]:

machine_hours = [[0, 0.7, 0.675, 0, 0.65 , 0.625, 0.7  ],
       [0.4  , 0.5  , 0.45 , 0.45 , 0.45 , 0.5  , 0.45 ],
       [0.375, 0.35 , 0.4  , 0.35 , 0.4  , 0.425, 0.35 ],
       [0.25 , 0.25 , 0.25 , 0.2  , 0.25 , 0.425, 0.4  ]]

#coefficients Cij
cost_production = [[ 0  , 17.4 , 17.4 ,  0  , 17.5 , 18.25, 19.75],
       [13.  , 14.1 , 14.22, 14.3 , 13.8 , 13.9 , 13.9 ],
       [10.65, 11.2 , 11.  , 11.25, 11.4 , 11.4 , 10.75],
       [ 9.6 ,  9.45,  9.5 ,  9.6 ,  9.6 ,  8.9 ,  9.4 ]]

#coefficients Tij
cost_transportation = [[0.3 , 0.4 , 0.8 , 0.7 , 0.7 , 0  , 0  ],
       [0.3 , 0.4 , 0.8 , 0.7 , 0.7 , 0.  , 0.5 ],
       [0.45, 0.6 , 1.2 , 1.05, 1.05, 0.  , 0.75],
       [0.45, 0.6 , 1.2 , 1.05, 1.05, 0.  , 0.75]]


capacity = [2500, 3000, 2500, 2600, 2500, 38000, 2500]
demand = [25000, 26000, 28000, 28000]

products = ['extra_fine', 'fine', 'medium', 'coarse']
suppliers = ['Ambrosi','Bresciani', 'Castri', 'De Blasi', 'Estensi', 'Filatoi R.', 'Giuliani']


In [3]:
l = len(products)
k = len(suppliers)
I = range(l)
J = range(k)

range(0, 7)


## Initiate the model

In [32]:
model = gp.Model('Filatoi Riuniti')

## Define the decision variables

In [33]:
x = model.addMVar((l,k), lb=0, vtype = GRB.CONTINUOUS, name = ['x_'+i+'_'+j for i in products for j in suppliers])

In [34]:
exp = sum(x[i][j]*(cost_production[i][j] + cost_transportation[i][j]) for i in I for j in J)

In [35]:
model.setObjective(exp, GRB.MINIMIZE)

## Define the contraints

## Demand constraints

In [36]:
for i in I:
    model.addConstr(sum(x[i][j] for j in J) >= demand[i], 'Demand_'+products[i])

## Capacity contraints

In [37]:
for j in J:
    model.addConstr(sum(machine_hours[i][j]*x[i][j] for i in I) <= capacity[j], 'Capacity_'+suppliers[j])

In [38]:
model.addConstr(x[0][0] == 0, name='Abrosi_restriction')

<(1,) matrix constraint *awaiting model update*>

In [39]:
model.addConstr(x[0][3] == 0, name='De Blasi_restriction')

<(1,) matrix constraint *awaiting model update*>

## Find the optimal solution

In [40]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 13 rows, 28 columns and 56 nonzeros
Model fingerprint: 0x6d49d343
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 4e+04]
Presolve removed 2 rows and 2 columns
Presolve time: 0.02s
Presolved: 11 rows, 26 columns, 52 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.650000e+04   0.000000e+00      0s
      14    1.3825443e+06   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.08 seconds
Optimal objective  1.382544334e+06


## Show the optimal solution

In [41]:
model.objVal

1382544.3343149226

## Show the optimal values for the DVs

In [42]:
print('Optimal values for DVs:')
for v in model.getVars():
    print(v.varName, '=', round(v.x,0))

Optimal values for DVs:
x_extra_fine_Ambrosi = 0.0
x_extra_fine_Bresciani = 4286.0
x_extra_fine_Castri = 3704.0
x_extra_fine_De Blasi = 0.0
x_extra_fine_Estensi = 3846.0
x_extra_fine_Filatoi R. = 13164.0
x_extra_fine_Giuliani = 0.0
x_fine_Ambrosi = 6250.0
x_fine_Bresciani = 0.0
x_fine_Castri = 0.0
x_fine_De Blasi = 0.0
x_fine_Estensi = 0.0
x_fine_Filatoi R. = 19750.0
x_fine_Giuliani = 0.0
x_medium_Ambrosi = 0.0
x_medium_Bresciani = 0.0
x_medium_Castri = 0.0
x_medium_De Blasi = 2040.0
x_medium_Estensi = 0.0
x_medium_Filatoi R. = 18817.0
x_medium_Giuliani = 7143.0
x_coarse_Ambrosi = 0.0
x_coarse_Bresciani = 0.0
x_coarse_Castri = 0.0
x_coarse_De Blasi = 0.0
x_coarse_Estensi = 0.0
x_coarse_Filatoi R. = 28000.0
x_coarse_Giuliani = 0.0


## (c)

## Generate the sensitivity analysis to determine if Falatoi Riuniti should rent the upgrade without re-optimizing the model from part (b)

In [44]:
print('Sensitivity Analysis (SA)\n ObjVal =', model.ObjVal)
model.printAttr(['X', 'Obj', 'SAObjLow', 'SAObjUp'])
model.printAttr(['X', 'RC', 'LB', 'SALBLow', 'SALBUp', 'UB', 'SAUBLow', 'SAUBUp'])
model.printAttr(['Sense', 'Slack', 'Pi', 'RHS', 'SARHSLow', 'SARHSUp']) # Pi = shadow price 
# NOTE: printAttr prints only rows with at least one NON-ZERO value, e.g. model.printAttr('X') prints only non-zero variable values

Sensitivity Analysis (SA)
 ObjVal = 1382544.3343149226

    Variable            X          Obj     SAObjLow      SAObjUp 
----------------------------------------------------------------
x_extra_fine_Ambrosi            0          0.3         -inf          inf 
x_extra_fine_Bresciani      4285.71         17.8         -inf      18.5735 
x_extra_fine_Castri       3703.7         18.2         -inf      19.4048 
x_extra_fine_De Blasi            0          0.7         -inf          inf 
x_extra_fine_Estensi      3846.15         18.2         -inf      18.9108 
x_extra_fine_Filatoi R.      13164.4        18.25      17.5392      20.0265 
x_extra_fine_Giuliani            0        19.75      17.9735          inf 
x_fine_Ambrosi         6250         13.3         -inf      13.6788 
x_fine_Bresciani            0         14.5       13.692          inf 
x_fine_Castri            0        15.02      14.0431          inf 
x_fine_De Blasi            0           15      14.9588          inf 
x_fine_Estensi 

## (d)

## No addtional code required. Question answered by refering to the senstivity analysis generated above for question (c)

## (e)

## Re-solve the model with an additional demand of 6,000 Kg/month such that demand for medium yarn increases from 28,000 Kg/month to 34,000 Kg/month 

In [ ]:
#import gurobipy as gp
#from gurobipy import GRB
#import numpy as np

In [29]:
machine_hours = [[0, 0.7, 0.675, 0, 0.65 , 0.625, 0.7  ],
       [0.4  , 0.5  , 0.45 , 0.45 , 0.45 , 0.5  , 0.45 ],
       [0.375, 0.35 , 0.4  , 0.35 , 0.4  , 0.425, 0.35 ],
       [0.25 , 0.25 , 0.25 , 0.2  , 0.25 , 0.425, 0.4  ]]

#coefficients Cij
cost_production = [[ 0  , 17.4 , 17.4 ,  0  , 17.5 , 18.25, 19.75],
       [13.  , 14.1 , 14.22, 14.3 , 13.8 , 13.9 , 13.9 ],
       [10.65, 11.2 , 11.  , 11.25, 11.4 , 11.4 , 10.75],
       [ 9.6 ,  9.45,  9.5 ,  9.6 ,  9.6 ,  8.9 ,  9.4 ]]

#coefficients Tij
cost_transportation = [[0.3 , 0.4 , 0.8 , 0.7 , 0.7 , 0  , 0  ],
       [0.3 , 0.4 , 0.8 , 0.7 , 0.7 , 0.  , 0.5 ],
       [0.45, 0.6 , 1.2 , 1.05, 1.05, 0.  , 0.75],
       [0.45, 0.6 , 1.2 , 1.05, 1.05, 0.  , 0.75]]


capacity = [2500, 3000, 2500, 2600, 2500, 38000, 2500]
demand = [25000, 26000, 34000, 28000]

products = ['extra_fine', 'fine', 'medium', 'coarse']
suppliers = ['Ambrosi','Bresciani', 'Castri', 'De Blasi', 'Estensi', 'Filatoi R.', 'Giuliani']


l = len(products)
k = len(suppliers)
I = range(l)
J = range(k)

## Initiate the model

model1 = gp.Model('Filatoi Riuniti with new client buying medium yarn')

## Define the decision variables

x = model1.addMVar((l,k), lb=0, vtype = GRB.CONTINUOUS, name = ['x_'+i+'_'+j for i in products for j in suppliers])

exp = sum(x[i][j]*(cost_production[i][j] + cost_transportation[i][j]) for i in I for j in J)

model1.setObjective(exp, GRB.MINIMIZE)

## Define the contraints

## Demand constraints

for i in I:
    model1.addConstr(sum(x[i][j] for j in J) >= demand[i], 'Demand_'+products[i])

## Capacity contraints

for j in J:
    model1.addConstr(sum(machine_hours[i][j]*x[i][j] for i in I) <= capacity[j], 'Capacity_'+suppliers[j])

model1.addConstr(x[0][0] == 0, name='Abrosi_restriction')

model1.addConstr(x[0][3] == 0, name='De Blasi_restriction')

<(1,) matrix constraint *awaiting model update*>

In [30]:
## Find the optimal solution

model1.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 13 rows, 28 columns and 56 nonzeros
Model fingerprint: 0xdb6ba533
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 4e+04]
Presolve removed 2 rows and 2 columns
Presolve time: 0.02s
Presolved: 11 rows, 26 columns, 52 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.725000e+04   0.000000e+00      0s
      12    1.4572379e+06   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.03 seconds
Optimal objective  1.457237883e+06


In [31]:
## Show the optimal solution

model1.objVal

1457237.882693438

In [38]:
## Show the minimum price that Filatoi Riuniti should quote to this new client 

price = round(((1457238 - 1382544)/6000),2)

print(price)

12.45


## (f)

## No addtional code required. Question answered by refering to the senstivity analysis generated above for question (c)

## (g)

## Re-solve the model with the new constraint that the production capacity of fine yarn at the Ambrosi mill is equal to 0.

In [19]:
#import gurobipy as gp
#from gurobipy import GRB
#import numpy as np

In [20]:
machine_hours = [[0, 0.7, 0.675, 0, 0.65 , 0.625, 0.7  ],
       [0.4  , 0.5  , 0.45 , 0.45 , 0.45 , 0.5  , 0.45 ],
       [0.375, 0.35 , 0.4  , 0.35 , 0.4  , 0.425, 0.35 ],
       [0.25 , 0.25 , 0.25 , 0.2  , 0.25 , 0.425, 0.4  ]]

#coefficients Cij
cost_production = [[ 0  , 17.4 , 17.4 ,  0  , 17.5 , 18.25, 19.75],
       [13.  , 14.1 , 14.22, 14.3 , 13.8 , 13.9 , 13.9 ],
       [10.65, 11.2 , 11.  , 11.25, 11.4 , 11.4 , 10.75],
       [ 9.6 ,  9.45,  9.5 ,  9.6 ,  9.6 ,  8.9 ,  9.4 ]]

#coefficients Tij
cost_transportation = [[0.3 , 0.4 , 0.8 , 0.7 , 0.7 , 0  , 0  ],
       [0.3 , 0.4 , 0.8 , 0.7 , 0.7 , 0.  , 0.5 ],
       [0.45, 0.6 , 1.2 , 1.05, 1.05, 0.  , 0.75],
       [0.45, 0.6 , 1.2 , 1.05, 1.05, 0.  , 0.75]]


capacity = [2500, 3000, 2500, 2600, 2500, 38000, 2500]
demand = [25000, 26000, 28000, 28000]

products = ['extra_fine', 'fine', 'medium', 'coarse']
suppliers = ['Ambrosi','Bresciani', 'Castri', 'De Blasi', 'Estensi', 'Filatoi R.', 'Giuliani']


l = len(products)
k = len(suppliers)
I = range(l)
J = range(k)

## Initiate the model

model2 = gp.Model('Filatoi Riuniti where Ambrosi produced no fine yarn')

## Define the decision variables

x = model2.addMVar((l,k), lb=0, vtype = GRB.CONTINUOUS, name = ['x_'+i+'_'+j for i in products for j in suppliers])

exp = sum(x[i][j]*(cost_production[i][j] + cost_transportation[i][j]) for i in I for j in J)

model2.setObjective(exp, GRB.MINIMIZE)

## Define the contraints

## Demand constraints

for i in I:
    model2.addConstr(sum(x[i][j] for j in J) >= demand[i], 'Demand_'+products[i])

## Capacity contraints

for j in J:
    model2.addConstr(sum(machine_hours[i][j]*x[i][j] for i in I) <= capacity[j], 'Capacity_'+suppliers[j])

model2.addConstr(x[0][0] == 0, name='Abrosi_restriction')

model2.addConstr(x[0][3] == 0, name='De Blasi_restriction')

model2.addConstr(x[1][0] == 0, name='New Ambrosi_restriction')

<(1,) matrix constraint *awaiting model update*>

In [21]:
## Find the optimal solution

model2.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 14 rows, 28 columns and 57 nonzeros
Model fingerprint: 0x497c0971
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 4e+04]
Presolve removed 3 rows and 3 columns
Presolve time: 0.05s
Presolved: 11 rows, 25 columns, 50 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.650000e+04   0.000000e+00      0s
      14    1.3849120e+06   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.06 seconds
Optimal objective  1.384911981e+06


In [22]:
## Show the optimal solution

model2.objVal

1384911.981373746

In [23]:
## Show the optimal values for the DVs

print('Optimal values for DVs:')
for v in model2.getVars():
    print(v.varName, '=', round(v.x,0))

Optimal values for DVs:
x_extra_fine_Ambrosi = 0.0
x_extra_fine_Bresciani = 4286.0
x_extra_fine_Castri = 3704.0
x_extra_fine_De Blasi = 0.0
x_extra_fine_Estensi = 3846.0
x_extra_fine_Filatoi R. = 13164.0
x_extra_fine_Giuliani = 0.0
x_fine_Ambrosi = 0.0
x_fine_Bresciani = 0.0
x_fine_Castri = 0.0
x_fine_De Blasi = 0.0
x_fine_Estensi = 0.0
x_fine_Filatoi R. = 26000.0
x_fine_Giuliani = 0.0
x_medium_Ambrosi = 6667.0
x_medium_Bresciani = 0.0
x_medium_Castri = 0.0
x_medium_De Blasi = 2726.0
x_medium_Estensi = 0.0
x_medium_Filatoi R. = 11464.0
x_medium_Giuliani = 7143.0
x_coarse_Ambrosi = 0.0
x_coarse_Bresciani = 0.0
x_coarse_Castri = 0.0
x_coarse_De Blasi = 0.0
x_coarse_Estensi = 0.0
x_coarse_Filatoi R. = 28000.0
x_coarse_Giuliani = 0.0


In [25]:
## calculate the difference between the new optimal solution and the optimal solution to the original model

1384912 - 1382544

2368

## (h)

## Re-solve the model with a new mill which represents overtime production at the Giuliani mill.

In [6]:
#import gurobipy as gp
#from gurobipy import GRB
#import numpy as np

## import problem data with the new model included

In [49]:
machine_hours = [[0, 0.7, 0.675, 0, 0.65 , 0.625, 0.7, 0.7],
       [0.4  , 0.5  , 0.45 , 0.45 , 0.45 , 0.5  , 0.45, 0.45],
       [0.375, 0.35 , 0.4  , 0.35 , 0.4  , 0.425, 0.35, 0.35],
       [0.25 , 0.25 , 0.25 , 0.2  , 0.25 , 0.425, 0.4, 0.4]]

#coefficients Cij
cost_production = [[ 0  , 17.4 , 17.4 ,  0  , 17.5 , 18.25, 19.75, 21.03],
       [13.  , 14.1 , 14.22, 14.3 , 13.8 , 13.9 , 13.9, 14.80 ],
       [10.65, 11.2 , 11.  , 11.25, 11.4 , 11.4 , 10.75, 11.45],
       [ 9.6 ,  9.45,  9.5 ,  9.6 ,  9.6 ,  8.9 ,  9.4, 10.01 ]]

#coefficients Tij
cost_transportation = [[0.3 , 0.4 , 0.8 , 0.7 , 0.7 , 0  , 0, 0],
       [0.3 , 0.4 , 0.8 , 0.7 , 0.7 , 0.  , 0.5, 0.5 ],
       [0.45, 0.6 , 1.2 , 1.05, 1.05, 0.  , 0.75, 0.75],
       [0.45, 0.6 , 1.2 , 1.05, 1.05, 0.  , 0.75, 0.75]]


capacity = [2500, 3000, 2500, 2600, 2500, 38000, 2500, 2500]
demand = [25000, 26000, 28000, 28000]

products = ['extra_fine', 'fine', 'medium', 'coarse']
suppliers = ['Ambrosi','Bresciani', 'Castri', 'De Blasi', 'Estensi', 'Filatoi R.', 'Giuliani', 'New Mill']

In [50]:
l = len(products)
k = len(suppliers)
I = range(l)
J = range(k)

## Initiate the model

In [51]:
model_overtime = gp.Model('Filatoi Riuniti with overtime production at Giuliani mill')

## Define the decision variables

In [52]:
x = model_overtime.addMVar((l,k), lb=0, vtype = GRB.CONTINUOUS, 
                           name = ['x_'+i+'_'+j for i in products for j in suppliers])

In [53]:
exp = sum(x[i][j]*(cost_production[i][j] + cost_transportation[i][j]) for i in I for j in J)

In [54]:
model_overtime.setObjective(exp, GRB.MINIMIZE)

In [55]:
## Define the contraints

## Demand constraints

for i in I:
    model_overtime.addConstr(sum(x[i][j] for j in J) >= demand[i], 'Demand_'+products[i])

## Capacity contraints

for j in J:
    model_overtime.addConstr(sum(machine_hours[i][j]*x[i][j] for i in I) <= capacity[j], 'Capacity_'+suppliers[j])

model_overtime.addConstr(x[0][0] == 0, name='Abrosi_restriction')

model_overtime.addConstr(x[0][3] == 0, name='De Blasi_restriction')

## Find the optimal solution

model_overtime.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 14 rows, 32 columns and 64 nonzeros
Model fingerprint: 0xdd2fd069
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [3e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 4e+04]
Presolve removed 2 rows and 2 columns
Presolve time: 0.06s
Presolved: 12 rows, 30 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.650000e+04   0.000000e+00      0s
      14    1.3823403e+06   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.12 seconds
Optimal objective  1.382340322e+06


In [56]:
## Show the optimal solution

model_overtime.objVal

1382340.3217697334

In [57]:
## Show the optimal values for the DVs

print('Optimal values for DVs:')
for v in model_overtime.getVars():
    print(v.varName, '=', round(v.x,0))

Optimal values for DVs:
x_extra_fine_Ambrosi = 0.0
x_extra_fine_Bresciani = 4286.0
x_extra_fine_Castri = 3704.0
x_extra_fine_De Blasi = 0.0
x_extra_fine_Estensi = 3846.0
x_extra_fine_Filatoi R. = 13164.0
x_extra_fine_Giuliani = 0.0
x_extra_fine_New Mill = 0.0
x_fine_Ambrosi = 6250.0
x_fine_Bresciani = 0.0
x_fine_Castri = 0.0
x_fine_De Blasi = 0.0
x_fine_Estensi = 0.0
x_fine_Filatoi R. = 19750.0
x_fine_Giuliani = 0.0
x_fine_New Mill = 0.0
x_medium_Ambrosi = 0.0
x_medium_Bresciani = 0.0
x_medium_Castri = 0.0
x_medium_De Blasi = 0.0
x_medium_Estensi = 0.0
x_medium_Filatoi R. = 18817.0
x_medium_Giuliani = 7143.0
x_medium_New Mill = 2040.0
x_coarse_Ambrosi = 0.0
x_coarse_Bresciani = 0.0
x_coarse_Castri = 0.0
x_coarse_De Blasi = 0.0
x_coarse_Estensi = 0.0
x_coarse_Filatoi R. = 28000.0
x_coarse_Giuliani = 0.0
x_coarse_New Mill = 0.0
